# Using the new `FluxCalculation` module

Now, the functions found in the various notebooks have been built into a few main modules. They include:
* calc.py - this contains the main calculations (ex. `vdk2016`, `lshell_to_glat`)
* data_access.py - this includes a helper function `grab_potsdam_file` which grabs the ap data file
* main.py - this contains the `FluxCalculation` class which brings this all together

Let's take a look at the "high level" api

## Imports

Before starting up this notebook, go to the main directory (one up from this one) and use


`pip install -e .` which install the `solarforcing` package to your environment.

In [1]:
from solarforcing import FluxCalculation
from datetime import datetime

### Investigate `FluxCalculation`

We see that this is full of defaults - feel free to change any of these by feeding in your desired value

In [2]:
FluxCalculation?

Init signature:
FluxCalculation(
    nbins_e: int = 128,
    min_e: float = 30.0,
    max_e: float = 1000.0,
    min_lshell: float = 2.0,
    max_lshell: float = 10.5,
    lshell_spacing: float = 0.5,
    data_dir: str = 'data/',
    ap_file_url: str = 'ftp://ftp.gfz-potsdam.de/pub/home/obs/Kp_ap_Ap_SN_F107/Kp_ap_Ap_SN_F107_since_1932.txt',
    start_date: datetime.datetime = None,
    end_date: datetime.datetime = None,
    angle: float = 80.0,
    standard_atmosphere_path: str = '../data/msis2.0_atm_out.txt',
) -> None
Docstring:      FluxCalculation(nbins_e: int = 128, min_e: float = 30.0, max_e: float = 1000.0, min_lshell: float = 2.0, max_lshell: float = 10.5, lshell_spacing: float = 0.5, data_dir: str = 'data/', ap_file_url: str = 'ftp://ftp.gfz-potsdam.de/pub/home/obs/Kp_ap_Ap_SN_F107/Kp_ap_Ap_SN_F107_since_1932.txt', start_date: datetime.datetime = None, end_date: datetime.datetime = None, angle: float = 80.0, standard_atmosphere_path: str = '../data/msis2.0_atm_out.txt')
File:

In [3]:
flux = FluxCalculation(start_date=datetime(2000, 1, 1), end_date=datetime(2014, 12, 31))

In [4]:
import urllib
import pandas as pd

### 1) Bringing in the data

In [5]:
flux.grab_data()

,ap
date,
2000-01-01,30.125
2000-01-02,16.500
2000-01-03,12.000
2000-01-04,12.875
2000-01-05,19.250
...,...
2014-12-27,6.250
2014-12-28,8.500
2014-12-29,20.875


### 2) Calculating the `vdk2016` flux

In [6]:
flux.calculate_flux()

array([[[2.28534529e-01, 2.21839226e-01, 2.15340073e-01, ...,
         5.55605092e-03, 5.39327708e-03, 5.23527197e-03],
        [6.85592696e-02, 6.66632579e-02, 6.48196807e-02, ...,
         2.05878145e-03, 2.00184570e-03, 1.94648451e-03],
        [4.45198300e-02, 4.33005557e-02, 4.21146739e-02, ...,
         1.38372591e-03, 1.34582951e-03, 1.30897098e-03],
        ...,
        [1.02388797e-01, 9.95163438e-02, 9.67244759e-02, ...,
         2.92078163e-03, 2.83884096e-03, 2.75919907e-03],
        [5.88091580e-02, 5.71894725e-02, 5.56143954e-02, ...,
         1.79199227e-03, 1.74263833e-03, 1.69464366e-03],
        [2.85765076e-02, 2.77974576e-02, 2.70396459e-02, ...,
         9.02608942e-04, 8.78002102e-04, 8.54066091e-04]],

       [[2.36234665e+00, 2.28459531e+00, 2.20940298e+00, ...,
         3.60205679e-02, 3.48350318e-02, 3.36885150e-02],
        [4.13578225e-01, 4.01749237e-01, 3.90258576e-01, ...,
         1.09959338e-02, 1.06814328e-02, 1.03759270e-02],
        [2.17886872e-01, 

### 3) Generate an `xarray.Dataset`

In [7]:
flux.generate_dataset()

<xarray.Dataset>
Dimensions:   (time: 5479, glat: 17, pressure: 101)
Coordinates:
  * glat      (glat) float64 44.71 50.53 54.53 57.51 ... 69.84 70.43 70.97 71.47
  * pressure  (pressure) float64 0.649 0.5603 0.4837 ... 3.086e-07 2.664e-07
  * time      (time) datetime64[ns] 2000-01-01 2000-01-02 ... 2014-12-31
Data variables:
    iprm      (time, glat, pressure) float64 0.001594 0.04147 ... 7.311e+10
    ap        (time) float64 30.12 16.5 12.0 12.88 ... 8.5 20.88 14.88 7.625


<xarray.Dataset>
Dimensions:              (time: 5479, lshell: 17, e: 128, glat: 17, pressure: 101)
Coordinates:
  * time                 (time) datetime64[ns] 2000-01-01 ... 2014-12-31
  * lshell               (lshell) float64 2.0 2.5 3.0 3.5 ... 8.5 9.0 9.5 10.0
  * glat                 (glat) float64 44.71 50.53 54.53 ... 70.43 70.97 71.47
  * e                    (e) float64 30.0 30.84 31.7 32.59 ... 946.3 972.8 1e+03
  * pressure             (pressure) float64 2.664e-07 3.086e-07 ... 0.5603 0.649
Data variables:
    vdk_energy_spectrum  (lshell, time, e) float64 0.2285 0.2218 ... 0.008446
    iprm                 (time, pressure, glat) float64 4.415e+06 ... 0.00284
    ap                   (time) float64 30.12 16.5 12.0 ... 20.88 14.88 7.625
Attributes:
    description:  Flux calculation
    units:        electrons / (cm2 s keV

## Skipping to the last step
You do not neccessarily need to call of this in order - for example, if you wanted to generate the dataset right away, you can use the following block of code

In [ ]:
flux = FluxCalculation()
flux.generate_dataset()